In [1]:
import yfinance as yf
from config import get_config_dict
import datetime as dt
import sys
from pathlib import Path
from loguru import logger
from tqdm import tqdm
import pandas as pd

from config import get_config_dict
from pipeline import Stock, Etl, market_scraper, Preprocess

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
handler = Etl()
#handler.extract_stock_data()
handler.extract_sp_500()
f_data, m_data = handler.load_stock_data()
index_data = handler.load_index_data()
print(f_data.shape, m_data.shape, index_data.shape)
preprocessor = Preprocess(f_data, m_data, index_data)
df = preprocessor.feature_engineering()

2023-08-19 17:19:45.710 | INFO     | pipeline.etl:extract_sp_500:34 - starting sp500 data extraction process
2023-08-19 17:19:46.899 | INFO     | pipeline.etl:load_stock_data:73 - loading stock data
2023-08-19 17:19:46.907 | INFO     | pipeline.stock:_initialize_stock:67 - repository for A exists (2023-05-23)
2023-08-19 17:19:46.923 | INFO     | pipeline.stock:_initialize_stock:67 - repository for AABA exists (2017-04-18)
2023-08-19 17:19:46.937 | INFO     | pipeline.stock:_initialize_stock:67 - repository for AAL exists (2023-07-20)


(197, 33) (12693, 4) (4687, 4)


KeyError: 0

In [3]:
import pandas as pd
import numpy as np
from utils import list_sp500_stocks

df =  pd.read_csv(
    '../data/1_work_data/SP500.csv',
    sep=';',
    usecols=['Symbol', 'Sector']
)


active_stocks = list_sp500_stocks()
active_stocks

,Symbol,Sector
0,MMM,Industrials
1,AOS,Industrials
2,ABT,Health Care
3,ABBV,Health Care
4,ACN,Information Technology
...,...,...
498,XYL,Industrials
499,YUM,Consumer Discretionary
500,ZBRA,Information Technology
501,ZBH,Health Care


In [6]:
all_df = pd.concat([df, active_stocks])
all_df = all_df.drop_duplicates(subset=['Symbol']).reset_index()
all_df['Status'] = np.where(all_df['Symbol'].isin(active_stocks['Symbol'].values.tolist()), True, False)
all_df.to_csv()

In [7]:
all_df

,index,Symbol,Sector,Status
0,0,A,Health Care,True
1,1,AABA,Financials,False
2,2,AAL,Industrials,True
3,3,AAP,Consumer Discretionary,False
4,4,AAPL,Information Technology,True
...,...,...,...,...
929,420,SOLV,Health Care,True
930,429,SMCI,Information Technology,True
931,457,UBER,Industrials,True
932,468,VLTO,Industrials,True
